In [52]:
from sklearn.model_selection import train_test_split 
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Load data and split into training and testing sets
df = pd.read_csv('cleveland_cleaned.csv')
X_train, X_test, y_train, y_test = train_test_split(df.drop('num', axis=1), df['num'], test_size=0.2, random_state=42)

# kNN with different values of k and distance weighting
k_values = [i for i in range(1, 20)]
f1ScoreMapping = {}
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')  # Using distance weighting
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    knn_f1 = f1_score(y_test, y_pred)

    f1ScoreMapping[k] = knn_f1


for k, f1 in f1ScoreMapping.items():
    print(f'k={k}: f1={f1:.4f}')

k=1: f1=0.7660
k=2: f1=0.7660
k=3: f1=0.8261
k=4: f1=0.8085
k=5: f1=0.7826
k=6: f1=0.7826
k=7: f1=0.7907
k=8: f1=0.8000
k=9: f1=0.8182
k=10: f1=0.8444
k=11: f1=0.8444
k=12: f1=0.8696
k=13: f1=0.8444
k=14: f1=0.8750
k=15: f1=0.8000
k=16: f1=0.8261
k=17: f1=0.8000
k=18: f1=0.8085
k=19: f1=0.7826


In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity

class KNNClassifier:
    def __init__(self, k=3, p=2):
        self.k = k
        self.p = p

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        predictions = []
        for i, x in X_test.iterrows():
            # Calculate Minkowski Distance between x and all training points
            distances = np.sum(np.abs(self.X_train - x) ** self.p, axis=1) ** (1 / self.p)
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = self.y_train.iloc[nearest_indices]
            unique_labels, counts = np.unique(nearest_labels, return_counts=True)
            most_common_label = unique_labels[np.argmax(counts)]
            predictions.append(most_common_label)


        return predictions

df = pd.read_csv('cleveland_cleaned.csv')
df_train = pd.DataFrame(df)
features = list(df_train.columns[:-1])
X_train = df_train[features]
y_train = df_train['num']
# print(X_train)
# print(y_train)

def trainKNN(X_train, y_train, k, p):
    # take 80% of the data as training data and 20% as testing data
    X_train, X_test, y_train, y_test = train_test_split(df_train.drop('num', axis=1), df_train['num'], test_size=0.2, random_state=42)

    knn = KNNClassifier(k=k, p=p)
    knn.fit(X_train, y_train)
    predictions = knn.predict(X_test)

    f1 = f1_score(y_test, predictions)
    return f1

scores = {}
for k in range(1, 20):
    for p in np.linspace(1, 2, 10):
        scores[(k, p)] = trainKNN(X_train, y_train, k, p)

top_kp = sorted(scores, key=scores.get, reverse=True)[:5]
for kp in top_kp:
    print(f'k={kp[0]}, p={kp[1]}: f1={scores[kp]:.4f}')

k=14, p=1.1111111111111112: f1=0.9130
k=14, p=1.2222222222222223: f1=0.9130
k=15, p=1.2222222222222223: f1=0.9130
k=15, p=1.3333333333333333: f1=0.9130
k=15, p=1.4444444444444444: f1=0.9130
